In [1]:
import cpp_soft_info

# Load Data

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from result_saver import SaverProvider
provider = SaverProvider()

In [5]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'

md = metadata_loader(_extract=True, _drop_inutile=False)
# print(md.head())
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]


md = md[md["meas_level"] == 1]
md = md[1:3]

d = max_distance
synd_rounds = max_distance

md

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,job_status,execution_date,extra,optimization_level,code,distance,rounds,logical,layout,descr
242,2023-10-30 09:43:20.984078+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpsa5vayrg008cpk30,NaN,{},[],1.0,True,single,...,JobStatus.DONE,2023-10-31 17:04:50.381134+01:00,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...
235,2023-10-30 09:42:31.121190+01:00,bigger_rep_codes,ibm_sherbrooke,cmzprxv3r3vg008wg7w0,NaN,{},[],1.0,True,single,...,JobStatus.DONE,2023-10-31 17:05:53.332697+01:00,NaN,NaN,RepetitionCodeCircuit,10.0,10,1,_is_hex=True,Run bigger Repetition codes v4: new distances ...


In [6]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()
    jobs[mmr_name] = job_id

memory_str = "mmr_log_0"
memory = memories[memory_str][:]
job_id = jobs[memory_str]
print(memory.shape)

(10000, 100)


# Initialize the code

In [7]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid


layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend(DEVICE), _is_hex=True)
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes
print("layout:", layout)
print("len(layout):", len(layout), " = distance * code qubits + (distance-1) * link qubits")

layout: [114, 116, 118, 120, 122, 104, 102, 100, 98, 113, 115, 117, 119, 121, 111, 103, 101, 99, 97]
len(layout): 19  = distance * code qubits + (distance-1) * link qubits


In [8]:
other_date = None # if none then it will find the closest to the tobecalib_job date
# other_date = "2023-11-10T10:30:00" # "2023-11-22T" works to
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 300, 2, other_date=other_date)

# takes 10s

Specified job execution date: 2023-10-31 17:04:50.381134+01:00
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_2pts_3std


#### Noise model stim

In [ ]:
from datetime import datetime

def get_avgs_from_dict(noise_dict):
    '''
    Takes noise dictionary as an argument then returns length 5
    array defining average noise levels: [init, idle, RO, single-, two-qubit gate]
    '''
    init_avg = np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    idle_avg = np.average([noise_dict[qubit]['idle'] for qubit in noise_dict])
    RO_avg = np.average([noise_dict[qubit]['RO'] for qubit in noise_dict])
    single_avg = 1e-4#np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    two_gate_avg = np.average([noise_dict[qubit]['2-gate'][connection]
                        for qubit in noise_dict
                        for connection in noise_dict[qubit]['2-gate']
                        if connection != 'default'])
    return [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]
# End get_avgs_from_dict


def get_noise_dict_from_backend(backend, layout, date=None):
    '''
    Takes a given backend and a mapping object and returns a noise dictionary
    to pass to a new Cross_Platform_Code/Hex_Code to have accurate backend
    specific noise information based on calibration data.
    '''
    # Pre-reqs
    noise_dict = {}
    round_time = 1000e-9
    if date is None:
        t = None
    else:
        t = datetime(day=int(date[-2:]), month=int(date[-4:-2]), year=int(date[:4]))
    properties = backend.properties(datetime=t)
    # Initializing each qubit with single qubit noise
    for qubit in layout:
        # Defining ROI error
        ROI_error = properties.readout_error(qubit)
        # Defining idle error
        t1 = properties.t1(qubit)
        t2 = properties.t2(qubit)
        t_time = min(t1, t2)
        idle_error = 1 - np.exp(-round_time / t_time)
        # Updating dictionary
        noise_dict[qubit] = {}
        noise_dict[qubit]['init'] = ROI_error
        noise_dict[qubit]['idle'] = idle_error
        noise_dict[qubit]['RO'] = ROI_error
        noise_dict[qubit]['gate'] = 1e-4
        noise_dict[qubit]['2-gate'] = {'default': 1e-2}
    # Two qubit gate noise
    all_qiskit_indexes = layout 
    for pair in backend.coupling_map:
        # Making sure connection is in circuit
        if pair[0] in all_qiskit_indexes and pair[1] in all_qiskit_indexes:
            # Getting two qubit gate error
            two_gate_error = properties.gate_error('ecr', pair)
            if two_gate_error > .5:
                two_gate_error = .5
            # # Updating dictionary
            # stim_index_0 = map.get_stim_index(qiskit_index=pair[0])
            # stim_index_1 = map.get_stim_index(qiskit_index=pair[1])
            noise_dict[pair[0]]['2-gate'][pair[1]] = two_gate_error
            noise_dict[pair[1]]['2-gate'][pair[0]] = two_gate_error
    return noise_dict
# End get_noise_dict_from_backend

In [ ]:

date = "2023-10-30 09:45:17.340386+01:00" #md.creation_date[0] # md[0] for log 1
noise_dict = get_noise_dict_from_backend(provider.get_backend(DEVICE), layout, date=None) # takes noise closest to today
print("noise_dict:", noise_dict)
avgs_noise = get_avgs_from_dict(noise_dict)
print("avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]:", avgs_noise)

# takes 4s

noise_dict: {114: {'init': 0.012899999999999912, 'idle': 0.004346196426116666, 'RO': 0.012899999999999912, 'gate': 0.0001, '2-gate': {'default': 0.01, 113: 0.0047563347934007905, 115: 0.009703241315324962}}, 116: {'init': 0.007000000000000006, 'idle': 0.0038247043118020407, 'RO': 0.007000000000000006, 'gate': 0.0001, '2-gate': {'default': 0.01, 115: 0.00393754614228195, 117: 0.0119017196933624}}, 118: {'init': 0.012299999999999978, 'idle': 0.003967615521162093, 'RO': 0.012299999999999978, 'gate': 0.0001, '2-gate': {'default': 0.01, 117: 0.006117238332777608, 119: 0.012474770199112584}}, 120: {'init': 0.009800000000000031, 'idle': 0.004334195683942554, 'RO': 0.009800000000000031, 'gate': 0.0001, '2-gate': {'default': 0.01, 119: 0.01096375917945358, 121: 0.00895255723476518}}, 122: {'init': 0.007900000000000018, 'idle': 0.00408632614978377, 'RO': 0.007900000000000018, 'gate': 0.0001, '2-gate': {'default': 0.01, 121: 0.007386859695892589, 111: 0.009660706469946018}}, 104: {'init': 0.00300

In [ ]:
from soft_info import draw_matching_graph



circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=d,
                                rounds=synd_rounds,
                                after_clifford_depolarization= avgs_noise[4], #two-qubit-fidelity,
                                after_reset_flip_probability= 0,# avgs_noise[0], #reset error,
                                before_measure_flip_probability= avgs_noise[2]*5, #measurement error,
                                before_round_data_depolarization= avgs_noise[1]) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

# Decode

In [ ]:
class sm:
    def __init__(self, qubit_mapping, grid_dict, processed_scaler_dict):
        self.qubit_mapping = qubit_mapping
        self.grid_dict = grid_dict
        self.processed_scaler_dict = processed_scaler_dict

simulator = sm(qubit_mapping, grid_dict, processed_scaler_dict)

IQ_data = memory

In [ ]:
import cpp_soft_info

_RESETS = False
LOGICAL = '0'
ROUNDS = synd_rounds

_DETAILED = False

matching = pymatching.Matching.from_detector_error_model(model)
# cpp_soft_info.reweight_edges_based_on_error_probs(matching._matching_graph, counts, False, "spitz")

result = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data,
                                           ROUNDS, int("0"), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                           common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                            p_offset = 1, p_multiplicator = 1)
print("num_errors:", result.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)

num_errors: 96 out of 10000 shots for _RESETS = False


In [ ]:
matching = pymatching.Matching.from_detector_error_model(model)

p_meas = -1
# p_meas = 16e-2 


result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_data[:],
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED)

print("num_errors:", result_informed.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
         
# takes 1s

num_errors: 263 out of 10000 shots for _RESETS = False


In [ ]:
matching = pymatching.Matching.from_detector_error_model(model)
result_flat = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED)
print("num_errors:", result_flat.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)


num_errors: 405 out of 10000 shots for _RESETS = False
